In [81]:
%matplotlib inline
import xml.etree.ElementTree as ET
from pylab import *
import networkx as nx
import pickle
import numpy as np
import math
import folium

In [82]:
def store(data, name):
    with open('../0_map_data_processed/'+name+'.p','wb') as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# Parsing the osm file using iterparse

In [83]:
ZOOM = 17
SCALE = (2**ZOOM)*(256)
R = 6378137
BANG_LAT = 12.9885#,77.6218
M_PER_PX = 156543.03392 * math.cos(BANG_LAT * (pi / 180)) / (2**ZOOM)
def getxy(lat,lon):
    # gets X,Y xoordinates using latitude and longitude
    scale = SCALE
    siny = math.sin(lat*(pi/180))
    siny = min(max(siny, -0.9999999999),0.99999999999)
    x = scale*(0.5+(lon/360))
    y = scale*(0.5-(math.log((1+siny)/(1-siny))/(4*pi)))
    return [x,y]

def node_attributes(elem):
    # Given a node element, gets its attributes like ID,latitude,longitude
    ID = elem.get('id')
    lat = float(elem.get('lat'))
    lon = float(elem.get('lon'))
    return ID,lat,lon

def way_attributes(elem):
    path = []
    ID = elem.get('id')
    for nd in elem:
        if nd.tag == "nd":
            path.append(nd.get('ref'))
    return ID,path  

def checkway(way,check):
    for nd in way:
        if(nd.tag == 'tag'):
            if(nd.attrib['k'] == check):
                return True
                break
    return False
        
def distance(x1,y1,x2,y2):
    return sqrt((x2-x1)**2 + (y2-y1)**2)
    
    
def parser(source):
    
    useful_nodes = {}
    node = {}               # {'node_ID1' : [x1,y1], 'node_ID2' : [x2,y2],.... }
    node_data = {}          # same as node but with lat and lon instead of x,y
    way = {}                # {'way_ID1' : [nodes 1], 'way_ID2' : [nodes 2],.... }
    G = []        # a list of edges - [(node1,node2,dist12),(node3,node4,dist34), ...]
    buildings = []
        
    for event, elem in ET.iterparse(source):
        if elem.tag == "node":
            ID,lat,lon = node_attributes(elem)
            node[ID] = getxy(lat,lon)
            node_data[ID] = [lat,lon]
            elem.clear()
        
        if elem.tag == "way":
            if(checkway(elem, "highway")):
                ID,path = way_attributes(elem)
                for i in range(len(path)-1):
                    [x1,y1] = node[path[i]]
                    [x2,y2] = node[path[i+1]]
                    d = distance(x1,y1,x2,y2)
                    G.append((path[i],path[i+1],d,len(G))) # a list of tuples
                    useful_nodes[path[i]] = True
                    useful_nodes[path[i+1]] = True
                way[ID] = path
                elem.clear()
            if(checkway(elem, "building")):
                ID,path = way_attributes(elem)
                path.pop()
                buildings.append(path)
                elem.clear()
        
    return node_data,node,useful_nodes,G,buildings

In [84]:
node_data,nodes,useful_nodes,G,buildings = parser('../0_map_data_raw/bangalore0')
print('Done')

Done


In [85]:
print("No of nodes - " , len(node_data))
print("No of edges - " , len(G))
print("No of buildings - ", len(buildings))
buildings[:5]

No of nodes -  3104016
No of edges -  505146
No of buildings -  587396


[['254438362', '254438363', '254438364', '254438365'],
 ['263870609', '263870628', '263870646', '263870666'],
 ['263871052', '263871073', '263871078', '263871083'],
 ['264111556', '264111557', '264111558', '264111559'],
 ['264112138', '264112139', '264112140', '264112141']]

In [86]:
def poly_area(x,y):
    x = np.array(x); y = np.array(y)
    x=x.astype(np.float32)
    y=y.astype(np.float32)
    x = x-np.min(x)
    y = y-np.min(y)
    correction = x[-1] * y[0] - y[-1]* x[0]
    main_area = np.dot(x[:-1], y[1:]) - np.dot(y[:-1], x[1:])
    return 0.5*np.abs(main_area + correction)*(M_PER_PX**2)

In [87]:
for x,y,d,i in G:
    x1,y1 = nodes[x]
    x2,y2 = nodes[y]
building_detail=[]    
for building in buildings:
    x_array=[]
    y_array=[]
    for ID in building:
        x1,y1 = nodes[ID]
        x_array.append(x1)
        y_array.append(y1)
    x_mid=sum(x_array)/len(x_array)
    y_mid=sum(y_array)/len(y_array)
    area =poly_area(x_array,y_array)
    building_detail.append([x_mid,y_mid,area])    

In [88]:
for i in range(len(buildings)):
    for ID in buildings[i]:
        if(ID == "5780899411"):
            print(i, buildings[i])
            print('')
            break

582127 ['5780899411', '5780899412', '5780899413', '5780899414', '5780899415', '5780899416', '5780899417']



In [89]:
print("building boundaries")
print(buildings[:5])
print("building details")
print(building_detail[:5])
print('no of buildings',len(building_detail))

building boundaries
[['254438362', '254438363', '254438364', '254438365'], ['263870609', '263870628', '263870646', '263870666'], ['263871052', '263871073', '263871078', '263871083'], ['264111556', '264111557', '264111558', '264111559'], ['264112138', '264112139', '264112140', '264112141']]
building details
[[24007848.483735323, 15558275.128534755, 9521.186267613708], [24016124.9407067, 15562255.782184906, 857.3130735987877], [24016190.788949333, 15562251.517115451, 1571.0634524085208], [24016097.819871005, 15562214.683060693, 744.9007748488676], [24016135.205100656, 15562214.056687979, 669.0563323188011]]
no of buildings 587396


In [90]:
building_no = 582127
print(node_data[buildings[building_no][0]])
test = folium.Map(location=node_data[buildings[building_no][0]], zoom_start=19)
for ID in buildings[building_no]:
    folium.Marker(node_data[ID], popup="here").add_to(test)    
print("area :",building_detail[building_no][2])
print("meters per px:",M_PER_PX)
test

[12.9190143, 77.6649113]
area : 1583.2527378151387
meters per px: 1.1637719042749568


## Extracting only Useful nodes from list of edges obtained

In [91]:
all_nodes = list(nodes.keys())
for ID in all_nodes:
    if(ID not in useful_nodes):
        del nodes[ID]
        del node_data[ID]

## Storing all the data for use from now on

In [92]:
print(len(nodes), len(node_data))

445653 445653


In [93]:
store(G, 'edges_list')
store(nodes, 'node_xy')
store(node_data, 'node_ll')
store(buildings, 'building_boundaries')
store(building_detail, 'building_details')